# This notebook works with the ARPrey project and it's current config.yaml file.

**Note**: It will also create the project if not already specified.
This notebook is specifically intended for tracking a mouse + two crickets in the VR Arena

In [ ]:
import deeplabcut as dlc
import random
import shutil
import glob
import os

# From paths.py
import paths

In [ ]:
# Find all ARPrey videos in the data directory
all_videos = glob.glob(paths.vrexperiment_path + os.sep + "*.avi")
ARPrey_videos = [p for p in all_videos if ('ARPrey' in p) and ('twocrickets' in p)]

## Load config file (if project already created)

In [ ]:
# Get config path
config_path = paths.config_path_vr_multi_cricket

# Read config file
cfg = dlc.auxiliaryfunctions.read_config(config_path)

# Get project directory
project_directory = cfg['project_path']

# Get list of local videos as found in the config
project_video_list = [vid for vid in cfg["video_sets"]]

## Pick videos not contained in the training set for testing

In [ ]:
# Randomly pick videos to test the network that are not part of the training set and copy to the project directory
test_video_dir = os.path.join(project_directory, "test-videos")
if not os.path.isdir(test_video_dir):
    os.mkdir(test_video_dir)

    # Randomly pick five videos outside of the training set to test the network on
    test_vids = []
    while(len(test_vids) < 5):
        vid = random.choice(ARPrey_videos)
        if vid not in project_video_list:
            dest = shutil.copy2(vid, test_video_dir)
            test_vids.append(dest)

else:
    test_vids = glob.glob(test_video_dir + os.sep + "*.avi")

## Create new project (only needs to be done once)

In [ ]:
dlc.create_new_project?

In [ ]:
# Randomly select 25 videos to train on
init_video_list = random.sample(ARPrey_videos, 25)
dlc.create_new_project("ARPrey", "Matt", init_video_list, 
                       working_directory=r"D:\dlc_vr_arena_twocrickets", 
                       multianimal=True,
                       copy_videos=True)

## Extract and label frames

In [ ]:
dlc.extract_frames?

In [ ]:
# will go through all of the videos on the list, so if many videos, it might take a while
dlc.extract_frames(config_path, mode='automatic', algo='kmeans', crop=False, userfeedback=False);

In [ ]:
# label frames
# generates folders in the labeled-data folder with the video name
dlc.label_frames(config_path)

In [ ]:
# check the labels
# generates folders in the labeled-data folder with the video name but the actual labels printed on the pics
dlc.check_labels(config_path);

## Create dataset and train network

In [ ]:
dlc.create_multianimaltraining_dataset?

In [ ]:
# create the training set
# also creates the Pose.yaml in the dlc-models/train dataset, which contains the setting for training, i.e. take a look
dlc.create_multianimaltraining_dataset(
    config_path,
    num_shuffles=1,
    net_type="dlcrnet_ms5",
)

In [ ]:
dlc.train_network?

In [ ]:
# train the network
dlc.train_network(config_path, 
                  displayiters=20000, 
                  saveiters=10000, 
                  maxiters=100000, 
                  allow_growth=True)

## Evaluate network

In [ ]:
dlc.evaluate_network?

In [ ]:
# evaluate the network
dlc.evaluate_network(config_path, plotting=True)

In [ ]:
dlc.extract_save_all_maps?

In [ ]:
dlc.extract_save_all_maps(config_path, shuffle=1, gputouse=0)

## Pick a test video for pose estimation before proceeding to tracklets

In [ ]:
vid_to_test = test_vids[0]

In [ ]:
dlc.create_video_with_all_detections?

In [ ]:
scorername = dlc.analyze_videos(config_path,[vid_to_test])
dlc.create_video_with_all_detections(config_path, [vid_to_test], videotype='.mp4')

In [ ]:
dlc.convert_detections2tracklets?

In [ ]:
dlc.convert_detections2tracklets(config_path, [vid_to_test], 
                                 shuffle=1, trainingsetindex=0, 
                                 track_method="ellipse", videotype='mp4', 
                                 overwrite=True)

In [ ]:
pickled_tracklet_file = r'D:\dlc_vr_arena_twocrickets\ARPrey-Matt-2021-09-14\test-videos\08_10_2021_12_24_45_ARPrey_DG_210203_a_succ_camo_crickettexture_twocrickets_capturelight_sl75_led25DLC_dlcrnetms5_ARPreySep14shuffle1_100000_el.pickle'

In [ ]:
dlc.utils.make_labeled_video.create_video_from_pickled_tracks?

In [ ]:
dlc.utils.make_labeled_video.create_video_from_pickled_tracks(vid_to_test,
                                                             pickled_tracklet_file,
                                                             pcutoff=0.3)

In [ ]:
dlc.refine_tracklets?

In [ ]:
dlc.refine_tracklets(config_path,
                     pickled_tracklet_file,
                     vid_to_test,
                     min_tracklet_len=0,
                    )

In [ ]:
dlc.stitch_tracklets(config_path, vid_to_test,
                     track_method="ellipse",
                     min_length=3,
)

## Analyze new videos

In [ ]:
dlc.analyze_videos?

In [ ]:
# Test one video
dlc.analyze_videos(config_path, test_vids, videotype='mp4')

In [ ]:
dlc.create_labeled_video?

In [ ]:
# use the path defined above
dlc.create_labeled_video(config_path, test_vids[0],
    color_by="individual",
    keypoints_only=False,
    trailpoints=10,
    draw_skeleton=False,
    track_method="ellipse",
)

In [ ]:
# extract outlier frames
dlc.extract_outlier_frames?

In [ ]:
# extract outlier frames
dlc.extract_outlier_frames(config_path, project_video_list, outlieralgorithm='uncerrtain', p_bound=0.8)

In [ ]:
dlc.refine_labels(config_path)

In [ ]:
# Merge the dataset, then move to check labels and retrain
dlc.merge_datasets(config_path)

## Add extra videos (if needed)

In [ ]:
dlc.extract_frames?

In [ ]:
# If adding new videos, do that here by randomly selecting some not in the original data set or the test set
extra_vids = []
while(len(extra_vids) < 5):
    vid = random.choice(all_videos)
    if vid not in project_video_list or test_vids:
        extra_vids.append(vid)
        
dlc.add_new_videos(config_path, extra_vids, copy_videos=True)